In [5]:
import re
import json
import pickle
import os
import sys
import requests
import logging
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from tqdm.auto import tqdm
import plotly.io as pio
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as F
import wandb
import plotly.express as px
import pandas as pd
import torch.nn.init as init
from pathlib import Path
from jaxtyping import Int, Float
from torch import Tensor
import einops
from collections import Counter
from datasets import load_dataset
import pandas as pd
from ipywidgets import interact, IntSlider
from process_tiny_stories_data import load_tinystories_validation_prompts, load_tinystories_tokens

pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

logging.basicConfig(format='(%(levelname)s) %(asctime)s: %(message)s', level=logging.INFO, datefmt='%I:%M:%S')
sys.path.append('../')  # Add the parent directory to the system path

import utils.haystack_utils as haystack_utils
from sparse_coding.train_autoencoder import AutoEncoder
from utils.autoencoder_utils import custom_forward, AutoEncoderConfig, evaluate_autoencoder_reconstruction, get_encoder_feature_frequencies, load_encoder
import utils.haystack_utils as haystack_utils

%reload_ext autoreload
%autoreload 2

In [22]:
model_name = "tiny-stories-2L-33M"
model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device,
)


Using pad_token, but it is not set yet.


Loaded pretrained model tiny-stories-2L-33M into HookedTransformer


In [33]:
@torch.no_grad()
def get_acts(prompt: str, model: HookedTransformer, encoder: AutoEncoder, cfg: AutoEncoderConfig):
    _, cache = model.run_with_cache(prompt, names_filter=cfg.encoder_hook_point)
    acts = cache[cfg.encoder_hook_point].squeeze(0)
    _, _, mid_acts, _, _ = encoder(acts)
    return mid_acts


def get_max_activations(prompts: list[str], model: HookedTransformer, encoder: AutoEncoder, cfg: AutoEncoderConfig):
    activations = []
    indices = []
    for prompt in tqdm(prompts):
        acts = get_acts(prompt, model, encoder, cfg)[:-1]
        value, index = acts.max(0)
        activations.append(value)
        indices.append(index)

    max_activation_per_prompt = torch.stack(activations)  # n_prompt x d_enc
    max_activation_token_index = torch.stack(indices)

    total_activations = max_activation_per_prompt.sum(0)
    print(f"Active directions on validation data: {total_activations.nonzero().shape[0]} out of {total_activations.shape[0]}")
    return max_activation_per_prompt, max_activation_token_index


def get_token_kurtosis_for_decoder(model: HookedTransformer, layer: int, decoder: torch.Tensor):
    '''Return excess kurtosis over all decoder features' cosine sims with the unembed (higher is better)'''
    W_out = model.W_out[layer]
    resid_dirs = torch.nn.functional.normalize(decoder @ W_out, dim=-1)
    unembed = torch.nn.functional.normalize(model.unembed.W_U, dim=0)
    cosine_sims = einops.einsum(resid_dirs, unembed, 'd_hidden d_model, d_model d_vocab -> d_hidden d_vocab')
    
    mean = einops.repeat(cosine_sims.mean(dim=-1), f'd_hidden -> d_hidden {cosine_sims.shape[1]}')
    std = einops.repeat(cosine_sims.std(dim=-1), f'd_hidden -> d_hidden {cosine_sims.shape[1]}')
    kurt = torch.mean((cosine_sims - mean / std) ** 4, dim=-1) - 3
    return kurt



In [32]:
torch.cuda.empty_cache()

In [3]:
# vocab_counts = get_occuring_tokens(model, load_tinystories_validation_prompts())
# percent_ever_occur = {vocab_counts.sum() / len(vocab_counts) * 100}
print(f'25% of tokens ever occur in the validation set')

25% of tokens ever occur in the validation set


In [4]:
# 1. List of clean features
# 2. Sort by indirect ablation increase
# 3. Sort for things in layer 1

In [7]:
l0_encoder, l0_config = load_encoder('18_morning_sun', model_name, model)
l1_encoder, l1_config = load_encoder('2_upbeat_snowball', model_name, model)
prompts = load_tinystories_validation_prompts()[:10000]
# l1_kurtosis = get_token_kurtosis_for_decoder(model, 1, l1_encoder.W_dec)
# px.histogram(pd.DataFrame({"kurtosis": l1_kurtosis.cpu()}))   

## Clean direction ablation

In [8]:
def get_direction_ablation_hook(encoder, direction, pos=None):
    def subtract_direction_hook(value, hook):
        x_cent = value[0, :] - encoder.b_dec
        acts = F.relu(x_cent @ encoder.W_enc[:, direction] + encoder.b_enc[direction])
        x_reconstruct = einops.einsum(acts, encoder.W_dec[direction, :], "pos, d_mlp -> pos d_mlp") + encoder.b_dec
        if pos is not None:
            value[:, pos, :] -= x_reconstruct[pos]
        else:
            value[:, :] -= x_reconstruct
        return value
    return subtract_direction_hook

def evaluate_direction_ablation(prompts: list[str], encoder: AutoEncoder, model: HookedTransformer, direction: int, cfg: AutoEncoderConfig, pos: None | int = None) -> float:
    
    original_losses = []
    ablated_losses = []
    encoder_hook_point = f"blocks.{cfg.layer}.{cfg.act_name}"
    for prompt in prompts:
        if pos is not None:
            original_loss = model(prompt, return_type="loss", loss_per_token=True)[0, pos]
        else:
            original_loss = model(prompt, return_type="loss")
        
        with model.hooks(fwd_hooks=[(encoder_hook_point, get_direction_ablation_hook(encoder, direction, pos))]):
            if pos is not None:
                ablated_loss = model(prompt, return_type="loss", loss_per_token=True)[0, pos]
            else:
                ablated_loss = model(prompt, return_type="loss")
        original_losses.append(original_loss.item())
        ablated_losses.append(ablated_loss.item())
    return np.mean(original_losses), np.mean(ablated_losses)

original_loss, ablated_loss = evaluate_direction_ablation(prompts[:2], l1_encoder, model, 0, l1_config, pos=None)
print(original_loss, ablated_loss)

1.1389805674552917 1.2179880142211914


## Max activating examples

In [34]:
max_activations_l0, max_activation_token_indices_l0 = get_max_activations(prompts[:5000], model, l0_encoder, l0_config)

  0%|          | 0/5000 [00:00<?, ?it/s]

Active directions on validation data: 16384 out of 16384


In [35]:
max_activations_l0.shape

torch.Size([5000, 16384])

## Cosine sims

In [9]:
W_out = model.W_out[0]
W_in = model.W_in[1]
# d_hidden d_mlp d_mlp d_resid
# d_resid d_mlp d_mlp d_hidden
cosine_sims = torch.nn.functional.normalize(l0_encoder.W_dec @ W_out, dim=-1) @ torch.nn.functional.normalize(W_in @ l1_encoder.W_enc, dim=0)
cosine_sims = torch.tril(cosine_sims)
print(cosine_sims.shape)

all_sims = cosine_sims.flatten().cpu()

torch.Size([16384, 16384])


In [10]:
px.histogram(all_sims[torch.randperm(len(all_sims))][:10_000])

In [11]:
values, indices = torch.topk(all_sims, 100)
print(values[:10])

In [13]:
def i_to_row_col(i: int, n_cols: int = len(cosine_sims)):
    row = i // n_cols
    col = i % n_cols
    return row, col

l0_dir, l1_dir = i_to_row_col(indices[0], len(cosine_sims))
print(l0_dir, l1_dir)

tensor(14478) tensor(11523)


In [37]:
def get_top_activating_examples_for_direction(prompts, direction, max_activations_per_prompt, max_activation_token_indices, k=10):
    activations = max_activations_per_prompt[:, direction]
    _, prompt_indices = activations.topk(10)
    top_prompts = [prompts[i] for i in prompt_indices]
    token_indices = max_activation_token_indices[prompt_indices, direction]
    return top_prompts, token_indices

top_prompts, top_prompt_token_indices = get_top_activating_examples_for_direction(prompts, l0_dir, max_activations_l0, max_activation_token_indices_l0)

Once upon a time, there was a boy named Timmy. Timmy was very excited because he was going to a special event at his school. He put on his best clothes and went to the event with his mom.

When they arrived, it started to rain. Timmy's mom didn't have an umbrella, so they got wet. Timmy was upset because he didn't want to be wet at the event. He asked his mom if they could go home, but his mom said they couldn't because they were already there.

Timmy saw a girl who also didn't have an umbrella. He remembered his mom had taught him to be polite, so he went over to her and shared his umbrella. The girl was very happy and thanked Timmy. Timmy felt better because he had helped someone and he didn't mind the rain anymore.


In [16]:
# ablation increase on next token
# ablation increase over the whole prompt
# Baselines: ablate random active features 

def evaluate_direction_ablation_single_prompt(prompt: str, encoder: AutoEncoder, model: HookedTransformer, direction: int, cfg: AutoEncoderConfig, pos: None | int = None) -> float:
    encoder_hook_point = f"blocks.{cfg.layer}.{cfg.act_name}"
    if pos is not None:
        original_loss = model(prompt, return_type="loss", loss_per_token=True)[0, pos]
    else:
        original_loss = model(prompt, return_type="loss")
    
    with model.hooks(fwd_hooks=[(encoder_hook_point, get_direction_ablation_hook(encoder, direction, pos))]):
        if pos is not None:
            ablated_loss = model(prompt, return_type="loss", loss_per_token=True)[0, pos]
        else:
            ablated_loss = model(prompt, return_type="loss")
    return original_loss.item(), ablated_loss.item()

In [39]:
for prompt, pos in zip(top_prompts, top_prompt_token_indices.tolist()):
    original_loss, ablated_loss = evaluate_direction_ablation_single_prompt(prompt, l0_encoder, model, l0_dir, l0_config, pos=pos)
    print(original_loss, ablated_loss)

6.624821186065674 7.009105205535889
0.946776270866394 3.7560923099517822
0.09626125544309616 0.1398785263299942
1.1685785055160522 2.676297903060913
0.3833664059638977 2.4273502826690674
11.21658706665039 10.466326713562012
3.92356014251709 6.456003665924072
0.04210013151168823 1.907275915145874
1.011767864227295 2.744246006011963
0.6568070650100708 2.9739489555358887


In [46]:
# Baseline
loss_increases = []
for prompt in tqdm(prompts[:100]):
    acts = get_acts(prompt, model, l0_encoder, l0_config)[:-1]
    pos = 20
    active_directions = torch.argwhere(acts[pos, :] > 0).flatten()
    direction = random.choice(active_directions)
    original_loss, ablated_loss = evaluate_direction_ablation_single_prompt(prompt, l0_encoder, model, direction, l0_config, pos=pos)
    loss_increases.append(ablated_loss - original_loss)
print(np.mean(loss_increases))
px.histogram(loss_increases)

  0%|          | 0/100 [00:00<?, ?it/s]

In [19]:
print(max_activations.shape)

torch.Size([10000])


In [18]:
_, max_activating_prompt_indices = torch.topk(max_activations, k=10)
print(max_activation_token_indices)

tensor([14471, 14471, 14471,  ..., 14471, 14471, 14471], device='cuda:0')


In [17]:
for i in range(10):
    prompt = max_activations[i]
    print(prompt)
    break
    max_activation_token_indices_l0[:10]

tensor(14.2319, device='cuda:0')


In [ ]:
# one prompt
# save direction activations
# get index of max direction activation per prompt from pre-existing data
# return loss per token, original and ablated
# index into loss with positions where directions active, calculate loss increase


def custom_forward(
    enc: AutoEncoder, x: Float[Tensor, "batch d_in"], neuron: int, activation: float
):
    x_cent = x - enc.b_dec
    acts = F.relu(x_cent @ enc.W_enc + enc.b_enc)
    acts[:, neuron] = activation
    x_reconstruct = acts @ enc.W_dec + enc.b_dec
    return x_reconstruct, acts


@torch.no_grad()
def evaluate_autoencoder_reconstruction_per_token(prompt: str, pos: int, autoencoder: AutoEncoder, cfg: AutoEncoderConfig, model: HookedTransformer, direction: int):
    
    def encode_activations_hook(value, hook):
        _, x_reconstruct, _, _, _ = autoencoder(value[:, pos])
        value[:, pos] = x_reconstruct
        return value

    reconstruct_hooks = [(f"blocks.{cfg.layer}.{cfg.act_name}", encode_activations_hook)]

    def zero_ablate_hook(value, hook):
        x_reconstruct, _ = custom_forward(autoencoder, value[:, pos], direction, 0)
        value[:, pos] = x_reconstruct
        return value
    
    zero_ablate_hooks = [(f"blocks.{cfg.layer}.{cfg.act_name}", zero_ablate_hook)]
    
    original_loss = model(prompt, return_type="loss", loss_per_token=True)[0, pos].item()
    with model.hooks(reconstruct_hooks):
        reconstruct_loss = model(prompt, return_type="loss", loss_per_token=True)[0, pos].item()
    with model.hooks(zero_ablate_hooks):
        zero_ablate_loss = model(prompt, return_type="loss", loss_per_token=True)[0, pos].item()
    
    return original_loss, reconstruct_loss, zero_ablate_loss


dirs = [i_to_row_col(i) for i in indices]
data = []
for l0_dir, l1_dir in tqdm(dirs[:15]):
    # max_activations_l0[:, l0_dir]
    values, prompt_indices = torch.topk(max_activations_l0[:, l0_dir], k=50)
    
    original_losses = []
    reconstruct_losses = []
    zero_ablate_losses = []
    for i in prompt_indices:
        prompt = prompts[i]
        pos_index = max_activation_token_indices_l0[i, l0_dir].item()
        original_loss, reconstruct_loss, zero_ablate_loss = evaluate_autoencoder_reconstruction_per_token(prompt, pos_index, l0_encoder, l0_config, model, l0_dir)
        original_losses.append(original_loss)
        reconstruct_losses.append(reconstruct_loss)
        zero_ablate_losses.append(zero_ablate_loss)
    data.append([l0_dir, l1_dir, np.mean(original_losses), np.mean(reconstruct_losses), np.mean(zero_ablate_losses)])
df = pd.DataFrame(data, columns=["l0_dir", "l1_dir", "original_loss", "reconstruct_loss", "zero_ablate_loss"])
    

  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
df["loss_increase"] = df["zero_ablate_loss"] - df["reconstruct_loss"]
df.sort_values("loss_increase", ascending=False, inplace=True)
df

,l0_dir,l1_dir,original_loss,reconstruct_loss,zero_ablate_loss,loss_increase
1,tensor(3078),tensor(2717),1.139115,1.449524,1.500091,0.050567
4,tensor(3084),tensor(555),1.577442,1.571623,1.600115,0.028491
3,tensor(6306),tensor(4666),1.218571,1.254769,1.273814,0.019044
2,tensor(12738),tensor(1970),1.065921,1.106483,1.125427,0.018944
12,tensor(9183),tensor(7899),1.170940,1.397148,1.412604,0.015456
9,tensor(5471),tensor(2663),0.781454,0.889481,0.900346,0.010865
14,tensor(15319),tensor(525),1.029166,1.284547,1.291323,0.006776
8,tensor(12974),tensor(7589),3.265839,4.267891,4.269192,0.001301
10,tensor(15192),tensor(12425),1.819522,1.998446,1.999414,0.000969
6,tensor(15339),tensor(9325),0.621281,0.651502,0.649526,-0.001977


In [ ]:
# Percentage increase
((df["zero_ablate_loss"] - df["reconstruct_loss"]) / df["reconstruct_loss"]).mean()


0.004270189322327611

In [ ]:
((df["reconstruct_loss"] - df["original_loss"]) / df["original_loss"]).mean()

0.10807562473493677

In [ ]:
l0_acts = []
l1_acts = []
for prompt in tqdm(prompts[:500]):
    l0_act = get_acts(prompt, model, l0_encoder, l0_config).cpu()
    l1_act = get_acts(prompt, model, l1_encoder, l1_config).cpu()
    l0_acts.append(l0_act)
    l1_acts.append(l1_act)
l0_acts = torch.cat(l0_acts, dim=0)
l1_acts = torch.cat(l1_acts, dim=0)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
l0_norm = l0_acts.abs().sum(-1).mean()
print(l0_norm)
l1_norm = l1_acts.abs().sum(-1).mean()
print(l1_norm)

tensor(35.4037)
tensor(88.1443)


In [ ]:
del l0_acts, l1_acts